In [28]:
!pip install torch
!pip install tensorflow
!pip install simpletransformers
!pip install datasets
!nvidia-smi
!pip install transformers[torch]
!pip install accelerate==0.27.2 -U
!pip install datasets

Tue Jul 16 09:47:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0              30W /  70W |   7001MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
import torch
import tensorflow as tf
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import string
import re

In [2]:
torch.cuda.is_available()

True

In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
class DataPreprocessor():
    def __init__(self):
        pass
    def handle_columns(self, dataframe, *columns):   # sütun isimlerini günceller eski isimleri kaldırır.
      i=0;
      for num in range(0, len(columns)+1):
        column_name = dataframe.keys().tolist()[i]
        dataframe.rename({column_name: columns[0][i]}, axis=1, inplace=True)
        i = i + 1
      return dataframe
    def preprocess(self, dataframe):                 # texti taglerden, uzun boşluklardan ve noktalama işaretlerinden arındırır.
      i=0
      X = dataframe["text"]
      for i in range(0, len(X)-1):
        X[i] = X[i].lower()
        X[i] = re.sub(r'<.*?>', '', X[i])
        X[i] = re.sub(r'\s+', ' ', X[i])
        for char in X[i]:
          ascii_val=ord(char)
          if 33 <= ascii_val <= 64:
                X[i] = X[i].replace(char, '')
        i = i + 1
      return X
    def handle_missing_values(self, dataframe):      # eğer boş veriler label ya da text varsa o satırları siler
        dataframe.dropna(inplace=True)
        return dataframe

In [5]:
from datasets import load_dataset
import pandas as pd

# Veri setini yükle
dataset = load_dataset("batubayk/TR-News")

# Dataset'i pandas DataFrame'e dönüştür
df = pd.DataFrame(dataset['train'])

# İlgili sütunları seç ve NaN değerleri çıkar
df = df[["content", "topic"]].dropna()

print(df.head())

                                             content    topic
0  Tuğba Özerk, “Duymayan Kalmasın” programında, ...   Fiskos
1  MHP Genel Başkanı Bahçeli, Manisa'nın Selendi ...  Türkiye
2  Rusya’nın Suriye’ye havadan müdahaleye başlama...    dunya
3  Galatasaray’da forvet transferinde halen somut...   futbol
4  İspanya'nın Marca Gazetesi'nde yayımlanan deme...     Spor


In [6]:
df.loc[df["topic"].isin(df["topic"].value_counts()[df["topic"].value_counts() < 1000].index),"topic"] = "Diğer"
df.loc[df["topic"]=="spor", "topic"] = "Spor"
df.loc[df["topic"]=="dunya", "topic"] = "Dünya"
df.loc[df["topic"]=="bilim_ve_teknoloji", "topic"] = "Teknoloji"
df.loc[df["topic"]=="futbol", "topic"] = "Spor"
df.loc[df["topic"]=="turkiye", "topic"] = "Türkiye"
df.loc[df["topic"]=="ekonomi", "topic"] = "Ekonomi"
df.loc[df["topic"]=="Fiskos", "topic"] = "Diğer"
df.loc[df["topic"]=="siyaset", "topic"] = "Siyaset"
df.loc[df["topic"]=="Sanat", "topic"] = "Kültür-Sanat"
df.loc[df["topic"]=="Futbol", "topic"] = "Spor"
df.loc[df["topic"]=="Yaşam", "topic"] = "İş-Yaşam"
df.loc[df["topic"]=="Türkiye", "topic"] = "Gündem"
df.loc[df["topic"]=="Dünya", "topic"] = "Gündem"
df.loc[df["topic"]=="Siyaset", "topic"] = "Gündem"
df.loc[df["topic"]=="Sağlık", "topic"] = "Diğer"
df.loc[df["topic"]=="Otomobil", "topic"] = "Diğer"
df.loc[df["topic"]=="Eğitim", "topic"] = "Diğer"


In [7]:
def sample_extractor(dataframe, *topics):
    # Take an initial sample from the dataframe
    old_topic = dataframe.sample(1)
    topics = topics[0]
    for topic in topics:
        if topic in dataframe['topic'].values:
            # Check if there are at least 500 rows for the given topic
            new_topic = dataframe[dataframe["topic"] == topic]
            if len(new_topic) >= 400:
                new_topic = new_topic.sample(400)
            else:
                new_topic = new_topic.sample(len(new_topic))  # Sample all available rows if fewer than 500
            old_topic = pd.concat([old_topic, new_topic], axis=0)

    return old_topic


In [8]:
new_df = sample_extractor(df, ["Diğer", "Spor", "Teknoloji", "Ekonomi", "Gündem", "İş-Yaşam", "Kültür-Sanat"])
new_df.head()

,content,topic
186271,CHP Genel Başkan Yardımcısı Aykut Erdoğdu düze...,Gündem
190416,Antalya Şubesi ile Akdeniz Üniversitesi Fen Fa...,Diğer
97239,Warwick Üniversitesi ve Johns Hopkins Tıp Fakü...,Diğer
15385,Garanti Bankası yoluna Garanti BBVA olarak dev...,Diğer
114498,Kamu kurum ve kuruluşlarının mal ve hizmetleri...,Diğer


In [9]:
df = new_df.reset_index().drop(labels=["index"], axis=1)
df.head()

,content,topic
0,CHP Genel Başkan Yardımcısı Aykut Erdoğdu düze...,Gündem
1,Antalya Şubesi ile Akdeniz Üniversitesi Fen Fa...,Diğer
2,Warwick Üniversitesi ve Johns Hopkins Tıp Fakü...,Diğer
3,Garanti Bankası yoluna Garanti BBVA olarak dev...,Diğer
4,Kamu kurum ve kuruluşlarının mal ve hizmetleri...,Diğer


In [10]:
import re
data_handler = DataPreprocessor()
print(df.head())
df = data_handler.handle_columns(df, ["text", "label"])
print(df.head())
df["text"] = data_handler.preprocess(df)
df = data_handler.handle_missing_values(df)
print(df.head())

                                             content   topic
0  CHP Genel Başkan Yardımcısı Aykut Erdoğdu düze...  Gündem
1  Antalya Şubesi ile Akdeniz Üniversitesi Fen Fa...   Diğer
2  Warwick Üniversitesi ve Johns Hopkins Tıp Fakü...   Diğer
3  Garanti Bankası yoluna Garanti BBVA olarak dev...   Diğer
4  Kamu kurum ve kuruluşlarının mal ve hizmetleri...   Diğer
                                                text   label
0  CHP Genel Başkan Yardımcısı Aykut Erdoğdu düze...  Gündem
1  Antalya Şubesi ile Akdeniz Üniversitesi Fen Fa...   Diğer
2  Warwick Üniversitesi ve Johns Hopkins Tıp Fakü...   Diğer
3  Garanti Bankası yoluna Garanti BBVA olarak dev...   Diğer
4  Kamu kurum ve kuruluşlarının mal ve hizmetleri...   Diğer
                                                text   label
0  chp genel başkan yardımcısı aykut erdoğdu düze...  Gündem
1  antalya şubesi ile akdeniz üniversitesi fen fa...   Diğer
2  warwick üniversitesi ve johns hopkins tıp fakü...   Diğer
3  garanti bankası yolun

In [11]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [12]:
label_encoder = preprocessing.LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'].tolist())

In [13]:
features, targets = df['text'], df['label']
train_features, test_features, train_targets, test_targets = train_test_split(
        features, targets,
        test_size=0.2,
        random_state=42,
        shuffle=True,
        stratify=targets
    )

In [14]:
model_args = ClassificationArgs()

model_args.num_train_epochs = 5
model_args.max_seq_length=500
model_args.reprocess_input_data= True
model_args.overwrite_output_dir= True
model_args.train_batch_size= 8
model_args.fp16=False
model_args.output_dir= "bert_model"
model_args.learning_rate = 3e-5
model_args.no_save= True
model_args.save_eval_checkpoints=False
model_args.save_model_every_epoch=False
model_args.save_optimizer_and_scheduler=False

In [14]:

#create model with turkish bert
#model_bert = ClassificationModel('bert',
 #                                'dbmdz/bert-base-turkish-uncased',
  #                               num_labels=7,
   #                              args=model_args)

def custom_standardization(input_string):
    no_uppercased = tf.strings.lower(input_string, encoding='utf-8')
    no_stars = tf.strings.regex_replace(no_uppercased, "\*", " ")
    no_repeats = tf.strings.regex_replace(no_stars, "devamını oku", "")
    no_html = tf.strings.regex_replace(no_repeats, "", "")
    no_digits = tf.strings.regex_replace(no_html, "\w*\d\w*","")
    no_punctuations = tf.strings.regex_replace(no_digits, f"([{string.punctuation}])", r" ")
    no_extra_space = tf.strings.regex_replace(no_punctuations, " +"," ")

    no_I = tf.strings.regex_replace(no_extra_space, "ı","i")
    no_O = tf.strings.regex_replace(no_I, "ö","o")
    no_C = tf.strings.regex_replace(no_O, "ç","c")
    no_S = tf.strings.regex_replace(no_C, "ş","s")
    no_G = tf.strings.regex_replace(no_S, "ğ","g")
    no_U = tf.strings.regex_replace(no_G, "ü","u")

    return no_U.numpy().decode("utf-8")

train_txt=train_features.apply(custom_standardization)
test_txt=test_features.apply(custom_standardization)

train=pd.concat([train_txt, train_targets], axis=1)
test=pd.concat([test_txt, test_targets], axis=1)

In [15]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-cased", num_labels=7)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

def tokenize_data(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = train_dataset.map(tokenize_data, batched=True)
tokenized_test = test_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/2240 [00:00<?, ? examples/s]

Map:   0%|          | 0/561 [00:00<?, ? examples/s]

In [27]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=10e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    logging_strategy="epoch"
)

In [24]:
from transformers import Trainer

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.352700,1.070856
2,0.300000,1.185969
3,0.196600,1.290413
4,0.097500,1.345324
5,0.040500,1.377508


TrainOutput(global_step=700, training_loss=0.19745959418160575, metrics={'train_runtime': 1152.1867, 'train_samples_per_second': 9.721, 'train_steps_per_second': 0.608, 'total_flos': 2943539070187200.0, 'train_loss': 0.19745959418160575, 'epoch': 5.0})

In [34]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [38]:
model.push_to_hub("BertModels/turkishBertClassificationModel")

HfHubHTTPError:  (Request ID: Root=1-66964da3-33ba117a22e0181a50e4b531;e5f2557f-43c1-447f-a145-5d40403e0a63)

403 Forbidden: You don't have the rights to create a model under the namespace "BertModels".
Cannot access content at: https://huggingface.co/api/repos/create.
If you are trying to create or update content,make sure you have a token with the `write` role.

In [30]:
from transformers import pipeline
clf = pipeline("text-classification", model=model, tokenizer=tokenizer)
answer = clf("login olmadıı yaa.")
print(answer)

[{'label': 'LABEL_5', 'score': 0.9410179853439331}]


In [33]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming 'test_dataset['text']' is a list of tokenized sequences
test_sequences = [tokenizer.texts_to_sequences([text]) for text in test_dataset['text']]
test_sequences_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

i = 0
score = 0
for seq in test_sequences_padded:
    seq = np.array([seq])  # Add batch dimension
    answer = clf.predict(seq)  # Use model's predict method
    predicted_label = np.argmax(answer, axis=-1)[0]  # Get the predicted label
    if str(test_dataset['label'][i]) == str(predicted_label):
        score += 1
    i += 1

print(f"Accuracy: %{score * 100 / len(test_dataset['label'])}")


AttributeError: 'BertTokenizerFast' object has no attribute 'texts_to_sequences'

In [32]:
i = 0
score = 0
for line in test_dataset['text']:
  answer = clf(line)
  if str(test_dataset['label'][i]) == answer[0]["label"][-1:]:
    score = score + 1
  else:
    continue
  i = i + 1

print(f"Accuracy: %{score*100/len(test_df)}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1978 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (1978) must match the size of tensor b (512) at non-singleton dimension 1

In [16]:
model_bert.train_model(train)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/280 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/280 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/280 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/280 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/280 [00:00<?, ?it/s]

(1400, 0.4642716197835814)

In [17]:
result, model_outputs, wrong_predictions = model_bert.eval_model(test)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

In [18]:
result

{'mcc': 0.7608972895851563, 'eval_loss': 0.9681472579638163}

In [19]:
predictions = model_outputs.argmax(axis=1)
actuals = test_targets.values

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

0.7950089126559715